<a href="https://colab.research.google.com/github/edmosym/skills-copilot-codespaces-vscode/blob/main/MechMat_Tensile_Testing_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd
import io

uploaded=files.upload()

#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/#

In [ ]:
name='PASCO_Tensile_Data.csv' # Type the name of the file given from the first cell.
# If you run the first cell more than once on a file with the same name, be sure
# to include the new name, which might include (1) or something like that.
#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/#
#https://www.askpython.com/python-modules/pandas/pandas-read-csv-with-headers

TensileDataFrame=pd.read_csv(io.BytesIO(uploaded[name]), header=0)
print(TensileDataFrame)

In [ ]:
TensileDataFrame.plot()
# Note that the x-axis will be useful for determining the StartIndex and EndIndex

In [ ]:
StartIndex=3500 #Enter the starting index from the x-axis from above plot to
#select appropriate starting point of data to select; You might run multiple
#times to get the range you desire.

EndIndex=11920 #Enter the ending point of data to select
force=np.array(TensileDataFrame['Force (N) Run #1'])
position=np.array(TensileDataFrame['Position (m) Run #1'])

selected_force=force[StartIndex:EndIndex]
selected_position=position[StartIndex:EndIndex]
plt.plot(selected_position,selected_force)
plt.title('Force vs Position - Sample #',fontsize=20)
plt.xlabel('Position (m)',fontsize=16)
plt.ylabel('Force (N)',fontsize=16)
plt.show()

In [ ]:
# Show Plot for Stress and Strain
CrossArea= 0.006*(0.02*0.0254) #m^2
InitialLength= 63.5/1000 #m
StressData=selected_force/CrossArea*1e-6 # Stress in units of MPa
StrainData=selected_position/InitialLength # Strain in dimensionless units

#Calculate the modulus of toughness
m_t=np.trapz(StressData*1e6,StrainData)
#https://numpy.org/doc/stable/reference/generated/numpy.trapz.html
print('Modulus of Toughness =',np.round(m_t*1e-3,1),'kPa')

#Estimate ultimate tensile strength
S_u=np.max(StressData)
print('Estimated Ultimate Tensile Strength =',np.round(S_u,1),'MPa')

plt.plot(StrainData,StressData)
plt.title('Stress vs Strain - Sample #',fontsize=16)
plt.xlabel('Strain',fontsize=16)
plt.ylabel('Stress (MPa)',fontsize=16)
plt.show()

In [ ]:
# Select and Show Data for Elastic Region

ElasticStartIndex=3500 #Enter the starting index from the x-axis from above plot to
#select appropriate starting point of data to select; You might run multiple
#times to get the range you desire.

ElasticEndIndex=6100 #Enter the ending point of data to select
selected_elastic_stress=force[ElasticStartIndex:ElasticEndIndex]/CrossArea*1e-6
selected_elastic_strain=position[ElasticStartIndex:ElasticEndIndex]/InitialLength

slope, intercept = np.polyfit(selected_elastic_strain, selected_elastic_stress, 1)
print('Elastic Modulus =', np.round(slope*1e-3,1), 'GPa')

#Calculate the modulus of resilience on the elastic region
m_r=np.trapz(selected_elastic_stress*1e6,selected_elastic_strain)
print('Modulus of Resilience =',np.round(m_r*1e-3,1),'kPa')

#Estimate the yield strength of the material (end of elastic region)
S_y=np.max(selected_elastic_stress)
print('Estimated Yield Strength =',np.round(S_y,1),'MPa')

plt.plot(selected_elastic_strain,selected_elastic_stress)
plt.title('Stress vs Strain for Elastic Region - Sample #',fontsize=16)
plt.xlabel('Strain',fontsize=16)
plt.ylabel('Stress (MPa)',fontsize=16)
plt.plot(selected_elastic_strain,selected_elastic_strain*slope+intercept)
plt.show()